In [1]:
from dotenv import load_dotenv

# .env 파일에 있는 환경변수들을 불러오기
load_dotenv()

True

In [2]:
# %pip install pypdf      # -q: 리스트 안 보여주고 조용히 설치
# %pip install pypdf langchain-community
# %pip install beautifulsoup4

## 저장 파트

In [3]:
# 1. Document Load (PDF)
# 지원하는 문서 로더: https://docs.langchain.com/oss/python/integrations/document_loaders
from langchain_community.document_loaders import PyPDFLoader

# 불러올 파일 위치
file_path = './nke-10k-2023.pdf'
#대상 pdf를 변환해줄 로더
loader = PyPDFLoader(file_path)
# 로더가 pdf를 python에서 쓸 수 있도록 변환(pdf 1page -> 1 Document)
docs = loader.load()

print(len(docs))    # 원본 pdf 페이지 수

c:\Users\sscbr\AppData\Local\Programs\Python\Python314\Lib\site-packages\langchain_core\_api\deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


107


In [6]:
# 2. Splitting
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True    # add_start_index : 원본 문서 내에서 chunk 시작위치를 metadata에 포함할지 결정
)

chunks = text_splitter.split_documents(docs)

print(len(chunks))   # 전체 chunk 개수
print(chunks[0])   # 첫번째 청크의 원본 텍스트 내용

516
page_content='Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
FOR THE FISCAL YEAR ENDED MAY 31, 2023
OR
☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
FOR THE TRANSITION PERIOD FROM                         TO                         .
Commission File No. 1-10635
NIKE, Inc.
(Exact name of Registrant as specified in its charter)
Oregon 93-0584541
(State or other jurisdiction of incorporation) (IRS Employer Identification No.)
One Bowerman Drive, Beaverton, Oregon 97005-6453
(Address of principal executive offices and zip code)
(503) 671-6453
(Registrant's telephone number, including area code)
SECURITIES REGISTERED PURSUANT TO SECTION 12(B) OF THE ACT:
Class B Common Stock NKE New York Stock Exchange
(Title of each class) (Trading symbol) (Name of each exchange on which registered)' metadata={

In [ ]:
# 3. Embedding(숫자로 바꾸기)
from langchain_openai import OpenAIEmbeddings

# 쪼개기
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

# 아래는 테스트용(실제 텍스트 -> 벡터로 바뀌는 과정)
v1 = embeddings.embed_query(chunks[0].page_content) # 청크1 벡터로 변환
v2 = embeddings.embed_query(chunks[1].page_content) # 청크2 벡터로 변환

# 차원수는 같아야 함
print(len(v1) == len(v2))   # True(embedding 길이는 둘 다 동일한 모델을 사용했기 때문에 같음)
print(v1[:10])  # 벡터 눈으로 확인하기

In [ ]:
# 4. Vector Store에 저장하기
from langchain_core.vectorstores import InMemoryVectorStore     # memory에서만 실행하는 임시 vector storage

# 테스트/개발용 메모리
vector_store = InMemoryVectorStore(embeddings)
# pdf 쪼개 놓은 chunks를 벡터스토어에 저장(저장 후 id 들이 나옴)
ids = vector_store.add_documents(chunks)

## 검색 파트

In [ ]:
# aadd(비동기식) vs add(기본)

# 벡터스토어 -> 검색기 활용
retriever = vector_store.as_retriever(
    search_type = 'similarity',         # 검색방식: 유사도
    search_kwargs = { 'k':3 }             # 결과개수: 3개
)

# 검색
retriever.invoke('나이키의 미국 영업점 개수는?')    # LangChain의 retriever 에서 내부적으로 질문을 임베딩 해줘서 따로 임베딩 안 해도 됨

## Agent에 통합

In [ ]:
# 검색기(retriever)를 Tool(함수)로 만들기
from langchain.tools import tool    # decorator

@tool   # LLM에게 "이 함수는 네가 필요할 때 직접 호출해서 쓸 수 있는 도구야" 라고 알려줌
# 검색어(query)를 인자로 받음
def search_vectorstore(query: str) -> str:
    """Retrieve info to help answer a query about Nike"""
    # 검색기 대신 벡터스토어 바로 활용하기 (chunk 2개만 검색)
    docs = vector_store.similarity_search(query, k=2)
    result = ''

    for doc in docs:
        result += doc.page_content + '\n\n'

    return result


print(search_vectorstore('나이키 영업점 개수'))

In [ ]:
from langchain.agents import create_agent
# agent 안에 prompt를 다 쓰면 길어지기 때문에 변수로 빼기
prompt = """너는 2023 나이키 10k 보고서를 검색하는 도구를 다룰 수 있어. 
사용자 질문에 답변하기 위해 필요하면 사용해. 경제분석 전문가처럼 답변해."""

agent = create_agent(
    model="openai:gpt-4.1-mini",
    tools=[search_vectorstore],
    system_prompt=prompt
)

content = "나이키 영업점 숫자와 각 영업점 평균 매출액이 궁금함."

agent.invoke(
    {
        "messages": [
            {"role": "user", "content": content}
        ]
    }
)

## Web 문서(HTML) RAG + Agent

In [7]:
# HTML은 문서 본문 외에 필요하지 않은 내용이 많음 -> 전처리 필요!
import bs4
from langchain_community.document_loaders import WebBaseLoader

# 전처리
bs4_strainer = bs4.SoupStrainer(class_=('post-title', 'post-header', 'post-content'))       # 해당 클래스만 뽑아옴
loader = WebBaseLoader(
    web_path="https://lilianweng.github.io/posts/2023-06-23-agent/",
    bs_kwargs={'parse_only': bs4_strainer}  # 처리기 넣기
    # web_paths=[]      # 여러개의 웹 문서를 가져올 때
)

docs = loader.load()
# 문서 페이지 수, 총 글자수
print(len(docs), len(docs[0].page_content))

USER_AGENT environment variable not set, consider setting it to identify your requests.


1 43047


In [8]:
# Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, chunk_overlap = 200, add_start_index = True
)

pieces = splitter.split_documents(docs)
# print(len(pieces))
# print(pieces[0])

# Embedding
from langchain_openai import OpenAIEmbeddings

t_embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

# Store
from langchain_core.vectorstores import InMemoryVectorStore

simple_vector = InMemoryVectorStore(t_embeddings)
t_ids = simple_vector.add_documents(pieces)

In [9]:
from langchain.tools import tool

# 함수 생성
@tool
def search_simple_vecotr(query: str) -> str:
    """Retrieve info to help answer a query about LLM"""
    # 검색기 대신 벡터스토어 바로 활용하기 (chunk 2개만 검색)
    t_docs = simple_vector.similarity_search(query, k=2)
    result = ''

    for doc in t_docs:
        result += doc.page_content + '\n\n'

    return result

# Agent에 통합하기
from langchain.agents import create_agent

t_prompt='LLM Agent는 어떤 방식으로 동작되는지 알려줘.'

t_agent = create_agent(
    model='openai:gpt-4.1-mini',
    tools=[search_simple_vecotr],
    system_prompt=t_prompt
)

t_content='LLM Agent는 어떤 방식으로 동작되는지 알려줘.'

# 에이전트가 *생각하고 도구를 사용하는 과정"을 실시간으로 생중계 해줌
for event in t_agent.stream({"messages": [{"role": "user", "content": t_content}]}, stream_mode='values'):
    event['messages'][-1].pretty_print()

================================ Human Message =================================

LLM Agent는 어떤 방식으로 동작되는지 알려줘.
================================== Ai Message ==================================
Tool Calls:
  search_simple_vecotr (call_noe3jQrqSV8Gxnxzp6boeA0w)
 Call ID: call_noe3jQrqSV8Gxnxzp6boeA0w
  Args:
    query: LLM Agent 동작 방식
================================= Tool Message =================================
Name: search_simple_vecotr

LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the age